# Exploration de la BDD 'nom_du_fichier.csv' et régression linéaire

## Étape 1 : Préparation et Importation des Données

In [ ]:
# 1. Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
# 2. Charger la base de données

df = pd.read_csv("lien vers le fichier")
df = pd.read_csv("lien vers le fichier",sep='\t') # synthaxe avec séparateur pour le cas des fichiers tsv

# regarder les histoires de décompression et le cas des fichiers stockés sur l'ordinateur

In [ ]:
# 3. Aperçu initial des données :

display(df.sample())  # Affiche 5 lignes aléatoirement
display(df.info())  # Donne des informations sur les types de colonnes et les valeurs manquantes
display(df.describe())  # Statistiques descriptives basiques (numériques uniquement)

# regarder l'option de normaliser sur describe

## Étape 2 : Inspection des Données

In [ ]:
# 1. Vérification des dimensions :

display(df.shape)  # Nombre de lignes et de colonnes

# 2. Recherche des valeurs manquantes :

display(df.isna().sum())  # Total des valeurs manquantes par colonne

In [ ]:
# Visualisation des données manquantes :

sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.show()

In [ ]:
# 3. Identification des doublons :

display(df.duplicated().sum())  # Nombre de lignes dupliquées

In [ ]:
# 4. Analyse des types de données :

display(df.dtypes)  # Vérification des types de colonnes

In [ ]:
# 5. Reformatage des colonnes au type (format) incorrect :

# Colonne(s) contenant des dates au mauvais format texte (object ou string) :

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')


# Colonne(s) catégorique mal classée comme object :

df[['colonne_categorielle','colonne_categorielle']] =
 df[['colonne_categorielle','colonne_categorielle']].astype('category')


# Colonne(s) de nombres mal classés comme object :

# SI les données contiennent des séparateurs de milliers ou d'autres caractères parasites :

df['colonne_numérique'] = df['colonne_numérique'].str.replace(',', '').astype(float) # Suppression des caractères spéciaux (comme les virgules)
df['colonne_numérique'] = pd.to_numeric(df['colonne_numérique'], errors='coerce') # Conversion directe si aucune transformation préalable n'est nécessaire


# Colonne(s) de booléen mal classés comme integer :

df['colonne_booleen'] = df['colonne_booleen'].astype('boolean')


# Colonne(s) d'identifiants mal classés comme numériques (exemple : codes postaux ou identifiants clients) :

df['id'] = df['id'].astype(str)
df['code_postal'] = df['code_postal'].astype(str)


## Étape 3 : Analyse des Variables

In [ ]:
# 1. Variables catégoriques :

# Distribution des catégories :

for col in df.select_dtypes(include=['category']):
    print(df[col].value_counts())
    sns.countplot(y=col, data=df)
    plt.show()

In [ ]:
# 2. Variables numériques :

# Statistiques de base et distribution :

for col in df.select_dtypes(include=['int64', 'float64']):
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribution de {col}")
    plt.show()


In [ ]:
# Boxplots pour détecter les outliers :

for col in df.select_dtypes(include=['int64', 'float64']):
    sns.boxplot(x=df[col])
    plt.title(f"Outliers pour {col}")
    plt.show()


# Étape 4 : Analyse des Relations

In [ ]:
# 1. Corrélation entre variables numériques :
# Matrice de corrélation :

correlation_matrix = df.select_dtypes('number').corr() # Calcul de la matrice
print(correlation_matrix)

sns.heatmap(correlation_matrix, cmap="coolwarm") # Affichage de la matrice
plt.title("Matrice de Corrélation")
plt.show()


In [ ]:
# 2. Relation entre variables numériques et catégoriques :

# Visualisation des moyennes par catégorie :

for col_cat in df.select_dtypes(include=['category']):
    for col_num in df.select_dtypes(include=['int64', 'float64']):
        sns.barplot(x=col_cat, y=col_num, data=df)
        plt.title(f"Moyenne de {col_num} par {col_cat}")
        plt.show()

In [ ]:
# Scatterplots pour observer les tendances :

# Récupérer les colonnes selon leurs types
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.difference(['colonne_a_predire'])
cat_cols = df.select_dtypes(include=['category']).columns

# Scatterplots pour les colonnes numériques uniquement
for var1 in num_cols:  # Colonnes numériques sans 'price'
    sns.scatterplot(x='colonne_a_predire', y=var1, data=df)
    plt.title(f"Scatterplot de {'colonne_a_predire'} et {var1}")
    plt.show()

# Parcourir toutes les combinaisons possibles de colonnes numériques et catégoriques
for var1 in num_cols:  # Colonnes numériques sans 'price'
    for var2 in cat_cols:  # Chaque colonne catégorielle
        # Scatterplot avec coloration par la variable catégorielle
        sns.scatterplot(x='colonne_a_predire', y=var1, hue=var2, data=df)
        plt.title(f"Scatterplot de {'colonne_a_predire'} et {var1} par {var2}")
        plt.show()

# Étape 5 : Gestion des Anomalies

In [ ]:
# 1. Traitement des valeurs manquantes :
# Vérifier les valeurs manquantes
# cf. étapes précédentes

# Supprimer ou imputer selon la situation :

# 1.1. Remplir les valeurs manquantes avec une statistique (moyenne, médiane, mode)

df['colonne'] = df['colonne'].fillna(df['colonne'].median())  # Exemple avec la médiane

# 1.2. Supprimer les lignes avec des valeurs manquantes si elles sont rares

df = df.dropna(subset=['colonne'], axis=0)

# 2. Traitement des outliers :

# 2.1. Identification des valeurs aberrantes avec l'IQR (Interquartile Range)

Q1 = df['colonne'].quantile(0.25)
Q3 = df['colonne'].quantile(0.75)
IQR = Q3 - Q1

# Définir les seuils pour détecter les valeurs aberrantes

seuil_bas = Q1 - 1.5 * IQR
seuil_haut = Q3 + 1.5 * IQR

# Afficher les lignes contenant des valeurs aberrantes

outliers = df[(df['colonne'] < seuil_bas) | (df['colonne'] > seuil_haut)]
print(outliers)

# Supprimer les valeurs aberrantes détectées

df = df[~((df['colonne'] < seuil_bas) | (df['colonne'] > seuil_haut))]

# 2.2. Vérification manuelle d'une donnée aberrante spécifique
# Exemple : Vérifier une valeur aberrante spécifique et afficher la ligne complète

specific_outlier_value = 999999
print(df[df['colonne'] == specific_outlier_value])

# Suppression manuelle de la ligne aberrante si confirmée comme erronée

df = df[df['colonne'] != specific_outlier_value]

# 2.3. Transformation des valeurs aberrantes (au lieu de les supprimer)
# Exemple : Remplacer les valeurs aberrantes par les limites supérieures ou inférieures
df['colonne'] = np.where(
    df['colonne'] < seuil_bas, seuil_bas,
    np.where(df['colonne'] > seuil_haut, seuil_haut, df['colonne'])
)

# Étape 6 : Régréssion linéaire multiple

In [ ]:
# 1. Choix des colonnes X pour la regression linéaire multiple :
# Extraire dynamiquement les noms des colonnes de type int, float et category
# et exclure les colonnes gênantes

columns_X0 = df.select_dtypes(include=['int64', 'float64','category']).columns.difference(['variable_a_predire','id','date']).tolist()

# Sélection des variables indépendantes (sans les valeurs manquantes)
df0 = df[df['variable_a_predire'].notna()]

X0 = df0[columns_X0]
y0 = df0['variable_a_predire']

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, train_size = 0.75)

print("Colonnes utilisées pour X :", columns_X0)


# 2. Entrainer le modèle

model0 = LinearRegression().fit(X0_train, y0_train)

print(f"Score d'entrainement : {model0.score(X0_train, y0_train)}")
print(f"Score de test : {model0.score(X0_test, y0_test)}")


# 3. Faire des prédictions

df_predict0 = df[df['variable_a_predire'].isna()]
predict0 = model0.predict(df_predict0[columns_X0])
print(f"Prédictions : {predict0}")